In [ ]:
# Завдання 2. VGG16 + Transfer Learning для CIFAR-10

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import callbacks  # <== Додано це!
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from sklearn.metrics import classification_report

# 1. Завантаження CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

# 2. Завантаження базової моделі VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

# 3. Замороження базових шарів (всі крім останніх 4-х)
for layer in base_model.layers[:-4]:
    layer.trainable = False

# 4. Додавання власних шарів поверх VGG16
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

model_vgg = Model(inputs=base_model.input, outputs=predictions)

# 5. Компіляція моделі
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Колбеки
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(factor=0.5, patience=3)

# 7. Навчання моделі
history_vgg = model_vgg.fit(x_train, y_train_cat, epochs=20, batch_size=64,
                            validation_split=0.2, callbacks=[early_stop, reduce_lr])

# 8. Summary моделі
model_vgg.summary()

# 9. Графіки втрат і точності
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history_vgg.history['loss'], label='train loss')
plt.plot(history_vgg.history['val_loss'], label='val loss')
plt.legend()
plt.title('Loss VGG16')

plt.subplot(1,2,2)
plt.plot(history_vgg.history['accuracy'], label='train acc')
plt.plot(history_vgg.history['val_accuracy'], label='val acc')
plt.legend()
plt.title('Accuracy VGG16')
plt.show()

# 10. Класифікаційний звіт
y_pred_vgg = model_vgg.predict(x_test)
y_pred_classes_vgg = np.argmax(y_pred_vgg, axis=1)

class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
print(classification_report(y_test, y_pred_classes_vgg, target_names=class_names))

# 11. Візуалізація розпізнавання
plt.figure(figsize=(10,10))
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(x_test[i])
    plt.title(f"True: {class_names[int(y_test[i])]}\nPred: {class_names[y_pred_classes_vgg[i]]}")
    plt.axis('off')
plt.show()


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/20
 17/625 ━━━━━━━━━━━━━━━━━━━━ 31:18 3s/step - accuracy: 0.1219 - loss: 2.4852